# Fundamentals of Statistics

## Checking model assumptions and assessment of goodness-of-fit

## Data

This dataset is from the [World Happiness Report](https://worldhappiness.report/ed/2018/). The report uses six key variables to measure happiness differences: “income, healthy life expectancy, having someone to count on in times of trouble, generosity, freedom and trust, with the latter measured by the absence of corruption in business and government.”
The Happiness Index is an indication of happiness based on survey results, that was first used in the 2012 World Happiness Report. In the survey, the respondents were asked to rate their happiness on a scale from 0 to 10. The Happiness Index is calculated by averaging the survey results of the respondents. 

We import the required modules and then read in the data which is in an Excel file. We make a subset of the data for the 2018, `Year == 2018`, and choose only the first 12 columns/variables. Since the data includes many missing values shown with NaN (Not a Number), the `dropna()` function is used to drop the rows where at least one element is missing.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
happyness = pd.read_excel("Chapter2OnlineData.xls")
happy_2018 = happyness.loc[(happyness['Year'] == 2018)] 
happy_2018 = happy_2018.iloc[0:136, 0:11]
happy_2018 = happy_2018.dropna()
happy_2018

## Linear Regression Model

### Regression model for Life Ladder as the response variable and Social Support as the predictor

The `lmplot` from Seaborn module makes a scatter plot with a regression line fitted to the variables. 

In [ ]:
sns.lmplot(data=happy_2018, x="Social support", y="Life Ladder", ci=None)
plt.title("Scatter plot with a regression line")
plt.show()

The function `ols` from `statsmodels.formula.api` is imported to fit the regression model mathematically and estimate the model parameters.

In [ ]:
from statsmodels.formula.api import ols
model_ss = ols('Q("Life Ladder") ~ Q("Social support")', data=happy_2018).fit()
model_ss.summary()

### Regression model for Life Ladder as the response variable and Perceptions of corruption as the predictor

We repeat the same process to fit another model and use Perceptions of corruption to predict the response variable.

In [ ]:
sns.lmplot(data=happy_2018, x="Perceptions of corruption", y="Life Ladder", ci=None)
plt.title("Scatter plot with a regression line")
plt.show()

The regression model is:

In [ ]:
from statsmodels.formula.api import ols
model_pc = ols('Q("Life Ladder") ~ Q("Perceptions of corruption")', data=happy_2018).fit()
model_pc.summary()

### Prediction using Regression

Let's say we have ten values for Social Support $0.0, 0.1, ..., 0.9, 1$ stored in a dataframe. We utilise the first regression model fitted before to predict values of the response variable, Life Ladder, for these Social Support values. The function for this purpose is `predict`.

In [ ]:
extradata = pd.DataFrame({"Social support": np.arange(0, 1.1, 0.1)})
predictions = model_ss.predict(extradata)
extradata["LLprediction"] = predictions
extradata

Plot the predicted values on the initial scatter plot.

In [ ]:
sns.lmplot(data=happy_2018, x="Social support", y="Life Ladder", ci=None)
plt.title("Scatter plot with a regression line")
sns.scatterplot(x="Social support", y="LLprediction", data=extradata, color="red", marker="s")
plt.show()

### Multiple linear regression (regression with more than one predictor)

It is quite common and necessary to fit a regression model with more than one predictor to include the effect of all the possible predictors on the response variable. The `scatter_matrix` below is an easy way of having a quick look at the relationship of all the variables of interest.

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(happy_2018[["Life Ladder", "Social support", "Perceptions of corruption"]])  
plt.show()

To fit a normal linear regression model with more than one predictor, we add `+` all the predictors while writing the `ols` model in the form of: 

`model_name = ols("y ~ x1 + x2 + x3 + x4", data=data_name).fit()`. 

In [ ]:
from statsmodels.formula.api import ols
model_sp = ols('Q("Life Ladder") ~ Q("Social support")+Q("Perceptions of corruption")', data=happy_2018).fit()
model_sp.params

In [ ]:
model_sp.summary()

### Model Residuals

The difference between a prediction and an observed response is a residual. The residuals of a regression model are either calculated directly or by using `.resid` after the fitted model. For the model of lafe ladder defined by the social support the residuals are:

In [ ]:
fitted = model_ss.fittedvalues
residuals = happy_2018["Life Ladder"] - fitted
print(residuals)

In [ ]:
residuals = model_ss.resid
print(residuals)

## Checking model assumptions

Scatter plot of the variables was shown before. 

To assess normality of the residuals, a `qqplot` is made. Here, the residuals seem to be sufficiently following a normal distribution.  

In [ ]:
from statsmodels.api import qqplot
qqplot(data=model_ss.resid, fit=True, line="45")
plt.show()

The plot of residuals against fitted values is almost randomly patterned around zero. Although there is a bit of a curvature (it may be a good idea to fit a quadratic model to this data).

In [ ]:
#plotting fitted values (predictions) of the model against the model's residuals
sns.residplot(data=happy_2018, x=model_ss.fittedvalues, y=model_ss.resid, lowess=True)
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.show()
#for a model with only one predictor, you could also use:
#sns.residplot(data=happy_2018, x="Social support", y="Life Ladder", lowess=True)

### Outliers

In a regression model, the plot of residuals against fitted values is randomly patterned around zero and residuals are normally distributed. Another important point about residuals is to make sure that there are not any outliers present.

One way of checking for outliers in the model is via __Cook’s distance__. It is used to indicate influential data points that are particularly worth checking for validity.  

In [ ]:
sns.residplot(data=happy_2018, x="Perceptions of corruption", y="Life Ladder", lowess=True)
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.show()

In [ ]:
# see a boxplot of the residuals
#residuals = model_pc.resid
#plt.boxplot(residuals)
#plt.show()

Cook's distance shows the influence of each observation on the fitted response values. They are calculated and sorted as below.

In [ ]:
summary_pc = model_pc.get_influence().summary_frame()
summary_pc

In [ ]:
print(summary_pc.sort_values("cooks_d", ascending = False))

As a rule of thumb, Cook’s distances bigger than 0.3 are problematic. Row 1280 belongs to Rwanda which is an outlier.

In [ ]:
summary_pc[summary_pc["cooks_d"] > 0.3]

In [ ]:
happy_2018.loc[1280]

The red line is the regression line after removing Rwanda, which is shifted upwards a bit compared to the initial model.

In [ ]:
happy_2018_woRwanda = happy_2018[happy_2018["Country name"] != "Rwanda"]
sns.regplot(data=happy_2018, x="Perceptions of corruption", y="Life Ladder", ci=None, line_kws={"color": "green"})
sns.regplot(data=happy_2018_woRwanda, x="Perceptions of corruption", y="Life Ladder", ci=None, line_kws={"color": "red"})
plt.show()

## Model Assessment - How good is the model fitting the observations? 

1) RMSE (Root Mean Squared Error)

A "typical" difference between a prediction and an observed response in a regression model. The model of Life Ladder and the Social Support is saved as `model_ss`. We can directly calculate the RMSE or use the function `mse_resid` and take the square root of it.

In [ ]:
residuals = model_ss.resid
sum_sq_error = np.sum(residuals**2)
mse = sum_sq_error / (len(residuals)-2)
rmse = np.sqrt(mse)
print("rmse :", rmse)

In [ ]:
mse = rmse**2
print(mse)

In [ ]:
# use the function mse_resid
mse = model_ss.mse_resid
rmse = np.sqrt(mse)
rmse

In [ ]:
# rmse for the model with Perceptions of Corruption as the predictor 
mse = model_pc.mse_resid
rmse = np.sqrt(mse)
rmse

In [ ]:
# rmse for the model with two predictors
mse = model_sp.mse_resid
rmse = np.sqrt(mse)
rmse

2) R-squared 

The proportion of the variance in the response variable that is predictable by the explanatory variable in the model.

In [ ]:
model_ss.rsquared

In a model with only one predictor, R-squared is the correlation between the two variables (response and predictor) to the power of two.

In [ ]:
import scipy.stats
correlation =scipy.stats.pearsonr(happy_2018["Social support"], happy_2018["Life Ladder"])[0]
correlation**2

In [ ]:
#R-squared for all three models
print(model_ss.rsquared)
print(model_pc.rsquared)
print(model_sp.rsquared)

# Exercises

1. Fit a simple linrear regression model for `Life Ladder` as the dependent variable and `Freedom to make life choices` as the predictor variable.

2. Check the assumptions of this regression model.

3. Check the goodness of fit of this model.